In [1]:
import xarray as xr
import numpy as np
from xarray.coders import CFDatetimeCoder

coder = CFDatetimeCoder(use_cftime=True)
nico2100 = xr.open_dataset('/Users/jonniebarnsley/Downloads/nico_2100.nc', decode_times=coder)
xyla2100 = xr.open_dataset(
    '/Users/jonniebarnsley/Downloads/UKESM1-0-LL_thermal_thermal_forcing_8km_x_60m_2100.nc', 
    decode_times=coder
    )

In [ ]:
xyla2100

<xarray.Dataset> Size: 142MB
Dimensions:               (y: 768, x: 768)
Coordinates:
  * x                     (x) float64 6kB 4e+03 1.2e+04 ... 6.132e+06 6.14e+06
  * y                     (y) float64 6kB 4e+03 1.2e+04 ... 6.132e+06 6.14e+06
    time                  object 8B 2100-01-01 00:00:00
Data variables: (12/30)
    thermal_forcing_0000  (y, x) float64 5MB ...
    thermal_forcing_0001  (y, x) float64 5MB ...
    thermal_forcing_0002  (y, x) float64 5MB ...
    thermal_forcing_0003  (y, x) float64 5MB ...
    thermal_forcing_0004  (y, x) float64 5MB ...
    thermal_forcing_0005  (y, x) float64 5MB ...
    ...                    ...
    thermal_forcing_0024  (y, x) float64 5MB ...
    thermal_forcing_0025  (y, x) float64 5MB ...
    thermal_forcing_0026  (y, x) float64 5MB ...
    thermal_forcing_0027  (y, x) float64 5MB ...
    thermal_forcing_0028  (y, x) float64 5MB ...
    thermal_forcing_0029  (y, x) float64 5MB ...

In [9]:
def regrid_to_bisicles(ds: xr.Dataset) -> xr.Dataset:

    bisicles_8km_x = np.arange(4.000e+3, 6.148e+06, 8.000e+3)
    bisicles_centered = bisicles_8km_x - bisicles_8km_x.mean()

    # pad until dataset exceeds the extent of bisicles
    N = 10 # any big enough number will do
    padded = ds.pad(x=(N, N), y=(N, N))

    x = ds.x
    dx = x[1] - x[0]
    padded_axis_vals = np.arange(x.min() - N*dx, x.max()+N*dx+1, dx)
    padded['x'] = padded_axis_vals
    padded['y'] = padded_axis_vals

    # interpolate onto centered bisicles grid
    ds_bisicles = ds.interp(x=bisicles_centered, y=bisicles_centered)

    # revert to un-centered grid
    ds_bisicles['x'] = bisicles_8km_x
    ds_bisicles['y'] = bisicles_8km_x

    return ds_bisicles

In [10]:
def separate_levels(ds: xr.Dataset) -> xr.Dataset:
    data = {}
    for i, z in enumerate(ds.z):
        data[f'thermal_forcing_00{i:02d}'] = ds.sel(z=z).thermal_forcing
    return xr.Dataset(data)

In [11]:
def rename_levels(ds: xr.Dataset) -> xr.Dataset:
    data = {}
    for i, z in enumerate(ds.z):
        data[f'thermal_forcing_00{i:02d}'] = ds[f'thermal_forcing00{i:02d}']
    return xr.Dataset(data, coords={'y': ds.y, 'x': ds.x})